In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import glob
import json

import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
from tqdm import tqdm
from google.colab.patches import cv2_imshow


#from detectron2.structures import BoxMode
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq '/content/drive/MyDrive/release_ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21.zip'

In [ ]:
!unzip -qq '/content/drive/MyDrive/Heat_Maps_UBPMC_Modified.zip'

unzip:  cannot find or open /content/drive/MyDrive/Heat_Maps_UBPMC_Modified.zip, /content/drive/MyDrive/Heat_Maps_UBPMC_Modified.zip.zip or /content/drive/MyDrive/Heat_Maps_UBPMC_Modified.zip.ZIP.


In [ ]:
BASE_DIR = '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/'
image_dir = BASE_DIR+'images/'
json_dir = BASE_DIR + 'annotations_JSON/'


In [ ]:
def check_valid_json_point(json_data, chart_type):

  if(json_data):
    if not ('task6'in json_data):
      return False
    if ('task6'in json_data):
      val = json_data['task6']
      #print(val)
      if(val is None):
        return False
      else:
        val = json_data['task6']['output']['visual elements']
        if(chart_type == 'line'):
          if('lines' in val):
            val = json_data['task6']['output']['visual elements']['lines']
            if(val is None):
               return False
          elif(chart_type == 'scatter'):
            if('scatter points' in val):
              val = json_data['task6']['output']['visual elements']['scatter']
              if(val is None):
                return False
          elif(chart_type == 'vertical box'):
            if('boxplots' in val):
              val = json_data['task6']['output']['visual elements']['boxplots']
              if(val is None):
                return False
  return True


In [ ]:
def get_paths_dataframe_point(charts ,root_dir,json_dir):
  paths_pd = {} #pd.DataFrame()
  img_list = []
  json_list = []
  img_path = []
  json_path = []
  single_bbox_data = []
  model_input = []
  for i in charts:
    path = root_dir + i
    img_list = sorted(os.listdir(path))
    json_p = json_dir + i
    json_list = sorted(os.listdir(json_p))
    for j,k in zip(img_list,json_list):
      # fi = image_path
      json_fd = open(json_p + '/' + k)
      json_data = json.load(json_fd)
      #print(json_p + '/' + k)
      if(check_valid_json_point(json_data, i)):
        json_single_path = json_p + '/' + k
        img_single_path = path + '/' + j
        #if(json_data['task6']!='Null'):
        img_path.append(img_single_path)
        json_path.append(json_single_path)
        json_fd = open(json_single_path)
        json_data = json.load(json_fd)
        #points = get_points(json_data)
        # if not points:
        #   print (json_data['task6']['input']['task1_output']['chart_type'])
        #   print("no points found in file ", json_single_path)
        # else:
        #   for point in points:
        #     model_input.append([img_single_path, point, j])
  
  paths_pd['images'] = img_path 
  paths_pd['json'] = json_path
  paths_pd['input'] = model_input
  print(paths_pd)
  #print(len(img_path))
  #print(len(json_path))
  return paths_pd

In [ ]:
def get_points_line(input):
  points = []
  class_no = 0
  for each_item in input:
    # class_no += 1    
    for point in each_item:
      x= point['x']
      y = point['y']
      points.append([x,y])
    
  return points


In [ ]:
def get_points_scatter(input):
  points = []
  class_no = 0
  for each_item in input:
    class_no += 1
    for point in each_item:
      print(point)
      x= point['x']

      y = point['y']
      
      #print([[x,y] , class_no])
      points.append([x,y])
    
  return points

In [ ]:
def get_points_boxplot(input):
  points = []
  element_types = ["first_quartile","max", "median", "min", "third_quartile" ]
  for element in input:
    for elem in element_types:  
      xc = element[elem]['x']
      yc = element[elem]['y']
  
      points.append([xc,yc])
  
  return points

In [ ]:
def get_points_gt(input):
  if(input['task6']['input']['task1_output']['chart_type'] == "line"):
    return get_points_line(input['task6']['output']['visual elements']['lines'])
  elif(input['task6']['input']['task1_output']['chart_type'] == "scatter"):
    print("Reached here")
    return get_points_scatter(input['task6']['output']['visual elements']['scatter points'])
  elif(input['task6']['input']['task1_output']['chart_type'] == "vertical box"):
    return get_points_boxplot(input['task6']['output']['visual elements']['boxplots'])


In [ ]:
def Gaussian_Mask(sigma, x, y, mask):
  for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
      Y = np.exp(- ( ((i-x)**2 + (j-y)**2)/(2*(sigma**2))))
      Y = Y *255
      if Y>= mask[i][j]:
        mask[i][j] = Y
  return mask  

In [ ]:
# json_fd = open(x)
# #print(self.json_path[index])
# json_data = json.load(json_fd)
# #print(json_data)

# points = get_points_gt(json_data)

# print(points)


# mask = np.zeros((1000,1000))

# # for point in points:
# mask = Gaussian_Mask(2, 500, 500, mask)



# mask = mask.astype(np.uint8)
# mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

# cv2_imshow(mask)
# cv2.waitKey(0)

In [ ]:
point_detector_folders = ["line", "scatter", "vertical_box" ]

In [ ]:
paths_Train = get_paths_dataframe_point(point_detector_folders,image_dir,json_dir)

{'images': ['/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1164420___1471-2458-5-51-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1183521___pgen.0010002.g005.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1185549___1471-2458-5-75-3.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1185549___1471-2458-5-75-4.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1190168___1471-2156-6-40-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1201368___pgen.0010033.g004.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1262190___pgen.0010051.g002.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1262190___pgen.0010051.g004.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1361804___1471-2458-5-130-4.jpg', '/content/ICPR2020_CHAR

In [ ]:
image_list_main = sorted(paths_Train['images'])

In [ ]:
print(len(image_list_main))

1244


In [ ]:
json_list_main = sorted(paths_Train['json'])

In [ ]:
print(len(json_list_main))

1244


In [ ]:
base_mask = '/content/Heat_Maps_UBPMC/'

In [ ]:
a = os.listdir(base_mask) 

FileNotFoundError: ignored

In [ ]:
print(a)

['scatter', 'vertical_box', 'line']


In [ ]:
def mask_paths(mask_dir):
  mask_list = []
  model_input = []
  for i in os.listdir(base_mask):
    path = base_mask + i
    img_list = sorted(os.listdir(path))
    for j in img_list:
        img_single_path = path + '/' + j
        mask_list.append(img_single_path)

    
  return mask_list

In [ ]:
mask = mask_paths(base_mask)

In [ ]:
mask = sorted(mask)

In [ ]:
print(mask)

['/content/Heat_Maps_UBPMC/line/PMC1112601___1471-2458-5-40-1.jpg', '/content/Heat_Maps_UBPMC/line/PMC1164420___1471-2458-5-51-1.jpg', '/content/Heat_Maps_UBPMC/line/PMC1183521___pgen.0010002.g005.jpg', '/content/Heat_Maps_UBPMC/line/PMC1185549___1471-2458-5-75-3.jpg', '/content/Heat_Maps_UBPMC/line/PMC1185549___1471-2458-5-75-4.jpg', '/content/Heat_Maps_UBPMC/line/PMC1190168___1471-2156-6-40-1.jpg', '/content/Heat_Maps_UBPMC/line/PMC1201368___pgen.0010033.g004.jpg', '/content/Heat_Maps_UBPMC/line/PMC1262190___pgen.0010051.g002.jpg', '/content/Heat_Maps_UBPMC/line/PMC1262190___pgen.0010051.g004.jpg', '/content/Heat_Maps_UBPMC/line/PMC1361804___1471-2458-5-130-4.jpg', '/content/Heat_Maps_UBPMC/line/PMC1459128___1471-2458-6-69-2.jpg', '/content/Heat_Maps_UBPMC/line/PMC1526726___1471-2458-6-167-7.jpg', '/content/Heat_Maps_UBPMC/line/PMC1712340___1471-2458-6-291-11.jpg', '/content/Heat_Maps_UBPMC/line/PMC1712340___1471-2458-6-291-2.jpg', '/content/Heat_Maps_UBPMC/line/PMC1781497___pgen.003

In [ ]:
print(image_list_main)

['/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1164420___1471-2458-5-51-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1183521___pgen.0010002.g005.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1185549___1471-2458-5-75-3.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1185549___1471-2458-5-75-4.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1190168___1471-2156-6-40-1.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1201368___pgen.0010033.g004.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1262190___pgen.0010051.g002.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1262190___pgen.0010051.g004.jpg', '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1361804___1471-2458-5-130-4.jpg', '/content/ICPR2020_CHARTINFO_UB_PM

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = DEVICE

In [ ]:
from PIL import Image, ImageOps

In [ ]:
class ChartDataset(Dataset):
  def __init__(self,image_list_main,mask, mode = 'box', image_size =416,transforms = None):
    self.image_path= image_list_main
    self.mask_path = mask
    self.data_len = len(self.image_path)
    self.transforms = transforms
    self.num_samples = self.data_len
    self.mode = mode

  def __getitem__(self, index):   
    single_image_name = self.image_path[index]

    #print(single_image_name)
    
    image = Image.open(single_image_name)
    image = image.convert("RGB")

    image_old_size = image.size

    if self.transforms:
      image = self.transforms(image)   

    mask_image_path= self.mask_path[index]
    #print(single_image_name)  
    mask_i = Image.open(mask_image_path)
    mask_i = mask_i.convert("RGB")

    if self.transforms:
      mask_i = self.transforms(mask_i)
  
    return image, mask_i 

  def __len__(self):
    return self.data_len

In [ ]:
train_transformers = torchvision.transforms.Compose([torchvision.transforms.Resize([256, 256]),torchvision.transforms.ToTensor()])

In [ ]:
train_dataset = ChartDataset(image_list_main,mask,transforms=train_transformers)

In [ ]:
train_load = DataLoader(train_dataset,batch_size = 8, shuffle = True,num_workers=20)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


outputchannels = 3

model = models.segmentation.deeplabv3_resnet101(pretrained=True,progress=True)
model.classifier = DeepLabHead(2048, outputchannels)
# model = UNET(in_channels=3, out_channels=18)
model = model.to(device)
loss_cross = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.0001,momentum = 0.9, weight_decay = 0.003)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.1, patience = 5, verbose = True)

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
def train_cust(model,train_load,loss_cross,number_of_epochs,optimizer,scheduler):
  
  device = DEVICE

  global train_loss
  global val_loss_l
  global train_acc
  global val_acc

  train_loss = [] 
  val_loss_l = []
  train_acc=[]
  val_acc = []
  previous_loss = 1000000

  for epoch in range(number_of_epochs):
    #print("Epoch number {}".format(epoch))
    model.train()
    count = 0
    rloss = 0
    rightin = 0
    val_loss = 0
    print("Epoch number {}".format(epoch+1))
    for data in train_load:
      #print(i)
      image_of_tensor,label = data
      image_of_tensor = image_of_tensor.to(device)
      label = label.to(device)
      # print(label)
      # count+=label.size(0)
      optimizer.zero_grad()
      output = model(image_of_tensor)
      # print(type(output))
      # print(output['out'])
      # print(output['out'].size)
      # print(output['out'].shape)
      # print(label.shape)
      loss = loss_cross(output['out'],label)
      # print(loss)
      # print()
      #print(label.shape)
      #print(output['out'].shape)
      loss.backward()
      optimizer.step()
      rloss += loss.item()
      #print(running_loss)
      # rightin = rightin + (label==predicted).sum().item()
      #print(running_correct)
    epoch_loss = rloss/len(train_load)
    scheduler.step(epoch_loss)
    train_loss.append(epoch_loss)
    # epoch_acc = rightin/count * 100
    # train_acc.append(epoch_acc)
    print("Epoch No is {} Training Loss at Epoch {}".format(epoch+1,epoch_loss))

    # val_loss,acc = evaluate_model(model,validation_load)

    # val_loss_l.append(val_loss)
    # val_acc.append(acc)

    # if val_loss>previous_loss+5:
    #   print("Early Stopping")

    #   break
    
    # previous_loss = val_loss


  return model,train_acc,val_acc,train_loss,val_loss_l

In [ ]:
epoch_1 =10

In [ ]:
model_1,train_acc_1,val_acc_1,train_loss_1,val_loss_l_1 = train_cust(model,train_load,loss_cross,epoch_1,optimizer,scheduler)

Epoch number 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch No is 1 Training Loss at Epoch 0.3532115743519404
Epoch number 2
Epoch No is 2 Training Loss at Epoch 0.1362705165759111
Epoch number 3
Epoch No is 3 Training Loss at Epoch 0.09810681607669745
Epoch number 4
Epoch No is 4 Training Loss at Epoch 0.08442781746196441
Epoch number 5
Epoch No is 5 Training Loss at Epoch 0.07796034651498
Epoch number 6
Epoch No is 6 Training Loss at Epoch 0.07406060106288163
Epoch number 7
Epoch No is 7 Training Loss at Epoch 0.07169161326228043
Epoch number 8
Epoch No is 8 Training Loss at Epoch 0.07011784634624536
Epoch number 9
Epoch No is 9 Training Loss at Epoch 0.06883148134996493
Epoch number 10
Epoch No is 10 Training Loss at Epoch 0.06790216508297583


In [ ]:
torch.save(model_1, 'model.pth')

In [ ]:
images = []
image = Image.open('/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg')
image = image.convert("RGB")
images.append(image)
image = Image.open('/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg')
image = image.convert("RGB")
images.append(image)
images = np.asarray(images)

images = train_transformers(images).cuda()   

main_y = model_1(image)

TypeError: ignored

In [ ]:
def evaluate_model(model,validation_load):
  ayyd_c = 0
  model.eval()
  main_prediction = 0
  main_vald = 0
  device = DEVICE
  with torch.no_grad():
    for data in validation_load:
      images,label = data
      images = images.to(device)
      label = label.to(device)
      main_vald += label.size(0)
      main_y = model(images)

      loss = loss_cross(main_y,label)
      
      _,predicted = torch.max(main_y.data,1)

      ayyd_c  += (predicted==label).sum().item() 

    acc = ayyd_c/main_vald * 100

  print("Accuracy is {}".format(acc))


  return loss,acc

In [ ]:
# print(x[-1])

In [ ]:
# x_sorted = sorted(x)

In [ ]:
# shape_1 = cv2.imread(y[0])

In [ ]:
# height, width, channels = shape_1.shape

In [ ]:
# print(height,width)

In [ ]:
# text_split = '/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg'

In [ ]:
# y = text_split.split('/')

In [ ]:
# print(y)

In [ ]:
# print(y[-1])

In [ ]:
# print(y[-2])

In [ ]:
# head, tail = os.path.split('/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/images/line/PMC1112601___1471-2458-5-40-1.jpg')

In [ ]:
# print(tail)

In [ ]:
# print(head+'/'+tail)

In [ ]:
# yy = tail.split

In [ ]:
# print(yy)

In [ ]:
folder_save = '/content/drive/MyDrive/Heat_Maps_1'

In [ ]:
print(image_list_main[0])

In [ ]:
image_read = cv2.imread(image_list_main[0])
height, width, channels = image_read.shape


In [ ]:
print(height,width)

In [ ]:
# image_l = image_list_main[0:2]
# json_l = json_list_main[0:2]
# for i in range(len(image_l)):
#   image_to_read = image_l[i]
#   print(image_to_read)
  # image_read = cv2.imread(image_to_read)
  # height, width, channels = image_read.shape

  # mask_shape = np.zeros((width,height))
  # json_fd = open(json_l[i])
  # json_data = json.load(json_fd)

  # points = get_points_gt(json_data)

  # for point in points:
  #   mask = Gaussian_Mask(2, point[0], point[1], mask_shape)
  
  # mask = mask.astype(np.uint8)
  # mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

  # head, tail = os.path.split(image_l[i])



  # filename_save = os.path.join(folder_save,'/'+tail)
  # cv2.imwrite(filename_save, mask)

In [ ]:
# image_l = image_list_main[0:2]
# json_l = json_list_main[0:2]
# for i in range(len(image_l)):
#   image_to_read = image_l[i]
#   image_read = cv2.imread(image_to_read)
#   height, width, channels = image_read.shape

#   mask_shape = np.zeros((height,width))
#   json_fd = open(json_l[i])
#   json_data = json.load(json_fd)

#   points = get_points_gt(json_data)

#   for point in points:
#     mask = Gaussian_Mask(2, point[1], point[0], mask_shape)
  
#   mask = mask.astype(np.uint8)
#   mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
#   imu = image_to_read.split('/')
#   chart_type = imu[-2]
#   image_na = imu[-1]
#   dirName = folder_save + '/' + chart_type
#   print(dirName)
#   if not os.path.exists(dirName):
#     os.makedirs(dirName)   
#   filename_save = dirName +'/'+image_na
#   cv2.imwrite(filename_save, mask)

In [ ]:
def mask_images_creation(image_list,json_list):
  image_l = image_list[1048:]
  json_l = json_list[1048:]
  for i in range(len(image_l)):
    image_to_read = image_l[i]
    image_read = cv2.imread(image_to_read)
    height, width, channels = image_read.shape
    #print(i)

    mask_shape = np.zeros((height,width))
    json_fd = open(json_l[i])
    json_data = json.load(json_fd)

    

    points = get_points_gt(json_data)
    if points is not None:
      for point in points:
        mask = Gaussian_Mask(2, point[1], point[0], mask_shape)
      
      mask = mask.astype(np.uint8)
      mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
      imu = image_to_read.split('/')
      chart_type = imu[-2]
      image_na = imu[-1]
      dirName = folder_save + '/' + chart_type
      #print(dirName)
      if not os.path.exists(dirName):
        os.makedirs(dirName)   
      filename_save = dirName +'/'+image_na
      cv2.imwrite(filename_save, mask)



In [ ]:
mask_images_creation(image_list_main,json_list_main)

In [ ]:
!unzip -qq '/content/drive/MyDrive/Heat_Maps_1.zip'

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = DEVICE

In [ ]:
class ChartDataset(Dataset):
  def __init__(self,paths,transforms = None):
    self.image_path= paths['images']
    self.image_ids = paths['image_ids']
    #self.extracted_data = paths['input']
    #print(self.image_path)
    self.json_path = paths['json'] 
    # self.to_tensor = torchvision.transforms.ToTensor()
    self.data_len = len(self.image_path)
    print(self.data_len)
    self.transforms = transforms
    self.num_samples = self.data_len
    


  def __getitem__(self, index):
   
    single_image_name = self.image_path[index]   
    image = Image.open(single_image_name)
    image = image.convert("RGB")
    image_old_size = image.size
    if self.transforms:
      image = self.transforms(image)

    image_new_size = image.size()
    
    

    #image = torchvision.transforms.ToTensor()(image)
    
    return image, target 

  def __len__(self):
    return self.data_len

In [ ]:
train_transformers = torchvision.transforms.Compose([torchvision.transforms.Resize([256, 256]),torchvision.transforms.ToTensor()])
train_dataset = ChartDataset(paths,transforms=train_transformers)
train_load = DataLoader(train_dataset,batch_size = 8, shuffle = True,num_workers=20)


In [ ]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


outputchannels = 2

model = models.segmentation.deeplabv3_resnet101(pretrained=True,progress=True)
model.classifier = DeepLabHead(2048, outputchannels)
# model = UNET(in_channels=3, out_channels=18)
model = model.to(device)
loss_cross = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum = 0.9, weight_decay = 0.003)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.1, patience = 5, verbose = True)

In [ ]:
def train_cust(model,train_load,loss_cross,number_of_epochs,optimizer,scheduler):
  
  device = DEVICE

  global train_loss
  global val_loss_l
  global train_acc
  global val_acc

  train_loss = [] 
  val_loss_l = []
  train_acc=[]
  val_acc = []
  previous_loss = 1000000

  for epoch in range(number_of_epochs):
    #print("Epoch number {}".format(epoch))
    model.train()
    count = 0
    rloss = 0
    rightin = 0
    val_loss = 0
    print("Epoch number {}".format(epoch+1))
    for data in train_load:
      #print(i)
      image_of_tensor,label = data
      image_of_tensor = image_of_tensor.to(device)
      label = label.to(device)
      # print(label)
      # count+=label.size(0)
      optimizer.zero_grad()
      output,x= model(image_of_tensor)
      loss = loss_cross(output,label)
      #print(label.shape)
      #print(output['out'].shape)
      loss.backward()
      optimizer.step()
      rloss += loss.item()
      #print(running_loss)
      # rightin = rightin + (label==predicted).sum().item()
      #print(running_correct)
    epoch_loss = rloss/len(train_load)
    scheduler.step(epoch_loss)
    train_loss.append(epoch_loss)
    # epoch_acc = rightin/count * 100
    # train_acc.append(epoch_acc)
    print("Epoch No is {} Training Loss at Epoch {} and Epoch accuracy is {}".format(epoch+1,epoch_loss,epoch_acc))

    # val_loss,acc = evaluate_model(model,validation_load)

    # val_loss_l.append(val_loss)
    # val_acc.append(acc)

    # if val_loss>previous_loss+5:
    #   print("Early Stopping")

    #   break
    
    # previous_loss = val_loss


  return model,train_acc,val_acc,train_loss,val_loss_l

In [ ]:
def evaluate_model(model,validation_load):
  ayyd_c = 0
  model.eval()
  main_prediction = 0
  main_vald = 0
  device = DEVICE
  with torch.no_grad():
    for data in validation_load:
      images,label = data
      images = images.to(device)
      label = label.to(device)
      main_vald += label.size(0)
      main_y = model(images)

      loss = loss_cross(main_y,label)
      
      _,predicted = torch.max(main_y.data,1)

      ayyd_c  += (predicted==label).sum().item() 

    acc = ayyd_c/main_vald * 100

  print("Accuracy is {}".format(acc))


  return loss,acc

In [ ]:
epoch_1 = 10

In [ ]:
model_1,train_acc_1,val_acc_1,train_loss_1,val_loss_l_1 = train_cust(model,train_load,loss_cross,epoch_1,optimizer,scheduler)

In [ ]:
# import cv2

In [ ]:
# json_fd = open(x)
# #print(self.json_path[index])
# json_data = json.load(json_fd)
# #print(json_data)

# points = get_points_gt(json_data)

# print(points)


# mask = np.zeros((1000,1000))

# for point in points:
#   mask = Gaussian_Mask(2, point[0], point[1], mask)



# mask = mask.astype(np.uint8)
# mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

In [ ]:
class ChartDataset(Dataset):
  def __init__(self,paths, mode = 'box', image_size =416,transforms = None):
    self.image_path= paths['images']
    #self.image_ids = paths['image_ids']
    #self.extracted_data = paths['input']
    #print(self.image_path)
    self.json_path = paths['json'] 
    # self.to_tensor = torchvision.transforms.ToTensor()
    self.data_len = len(self.image_path)
    print(self.data_len)
    self.transforms = transforms
    self.num_samples = self.data_len
    self.mode = mode


  def __getitem__(self, index):
    #   if(self.mode == 'box'):
  #     return self.getitem_box(self,index)
  #   if(self.mode == 'point'):
  #     return self.getitem_point(self,index)
    

  # # def getitem_point(index):
  #   pass


  # def getitem_box(index):
    
    single_image_name = self.image_path[index]

    #print(single_image_name)
    
    image = Image.open(single_image_name)
    image = image.convert("RGB")
    # if self.transforms:
    #   img_as_img = self.transforms(img_as_img)
    # image = cv2.imread(single_image_name,cv2.IMREAD_COLOR)
    # image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # image= image/255.0
    image_old_size = image.size
    w, h = image.size
    #print(image_old_size)
    if self.transforms:
      image = self.transforms(image)

    image_new_size = image.size()
    #print(image_new_size)
    #img_height = image.shape[0]

    json_fd = open(self.json_path[index])
    #print(self.json_path[index])
    json_data = json.load(json_fd)
    #print(json_data)

    points = get_points_gt(json_data)

    
    mask = np.zeros((1000,1000))

    for point in points:
      mask = Gaussian_Mask(2, point[0], point[1], mask)

    
    mask = mask.astype(np.uint8)
    mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

    # cv2_imshow(mask)
    # cv2.waitKey(0)
    # if(points == None):
    #   points = [ [0,0], 0]
    #points = torch.FloatTensor(points)
    return image, points


  def __len__(self):
    return self.data_len

In [ ]:
# a = np.zeros((512,512))

In [ ]:
# point = [[200,200]]

In [ ]:
#  mask = Gaussian_Mask(2, 100, 100, a)

In [ ]:
# mask = a.astype(np.uint8)

In [ ]:
# mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

In [ ]:
# cv2_imshow(mask)
# cv2.waitKey(0)

In [ ]:
point_detector_folders = ["line", "scatter", "vertical_box" ]
paths_Train = get_paths_dataframe_point(point_detector_folders,image_dir,json_dir)

In [ ]:
transforms =  T.Compose([
                   T.Resize((416,416)),
                   T.ToTensor(), 
        #            T.Normalize(
        # mean=[0.485, 0.456, 0.406],
        # std=[0.229, 0.224, 0.225],)
                   ])


train_img_dataset = ChartDataset(paths_Train, mode = 'box' , image_size = 416, transforms = transforms)
#train_img_dataset_point = ChartDataset(paths_point, mode = 'point' , image_size = 416, transforms = transforms)

batch_size = 64

train_data_loader = torch.utils.data.DataLoader( train_img_dataset,shuffle = True,
     batch_size=batch_size, collate_fn=lambda x: x)

In [ ]:
print(train_data_loaser)

In [ ]:
for data in train_data_loader:
  print(data[0][1])
  break;

In [ ]:
# def get_balloon_dicts(img_dir):
#     json_file = os.path.join(img_dir, "via_region_data.json")
#     with open(json_file) as f:
#         imgs_anns = json.load(f)

#     dataset_dicts = []
#     for _, v in imgs_anns.items():
#         record = {}
        
#         filename = os.path.join(img_dir, v["filename"])
#         height, width = cv2.imread(filename).shape[:2]
        
#         record["file_name"] = filename
#         record["height"] = height
#         record["width"] = width
      
#         annos = v["regions"]
#         objs = []
#         for _, anno in annos.items():
#             assert not anno["region_attributes"]
#             anno = anno["shape_attributes"]
#             px = anno["all_points_x"]
#             py = anno["all_points_y"]
#             poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
#             #print("poly1", poly)
#             poly = list(itertools.chain.from_iterable(poly))
#             #print("poly2", poly)
#             obj = {
#                 "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
#                 "bbox_mode": BoxMode.XYXY_ABS,
#                 "segmentation": [poly],
#                 "category_id": 0,
#                 "iscrowd": 0
#             }
#             print(obj["segmentation"])
#             objs.append(obj)
#         record["annotations"] = objs
#         dataset_dicts.append(record)
#     return dataset_dicts

# from detectron2.data import DatasetCatalog, MetadataCatalog
# for d in ["train"]:
#     #DatasetCatalog.register("balloon/" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
#     print(get_balloon_dicts(("balloon/" + d)))
#     #MetadataCatalog.get("balloon/" + d).set(thing_classes=["balloon"])
# #balloon_metadata = MetadataCatalog.get("balloon/train")





In [ ]:
# # write a function that loads the dataset into detectron2's standard format

# def get_charts_dicts(paths):
#   image_paths = paths['images']
#   json_paths = paths['json']
#   image_ids = paths['image_ids']

#   dataset_dicts = []

#   for image_path,json_path, image_id in zip(image_paths, json_paths, image_ids):
    
#     image = Image.open(image_path)
#     image = image.convert("RGB")
#     width, height = image.size

#     json_fd = open(json_path)
#     json_data = json.load(json_fd)

#     record = {}
#     record["file_name"] = image_path
#     record["height"] = height
#     record["width"] = width
#     objs = []
#     bboxes = get_bounding_box_gt(json_data, image.size, image.size)
#     for box in bboxes[0]: 
#       poly = [box[0], box[1], box[2], box[1], box[2], box[3], box[0], box[3]]
#       obj = {
#           "bbox":box,
#           "bbox_mode": BoxMode.XYXY_ABS,
#           "segmentation": [poly],
#           "category_id": 0,
#           "iscrowd": 0
#       }
#       objs.append(obj)
      
#     record["annotations"] = objs
#     dataset_dicts.append(record)

#   return dataset_dicts


In [ ]:
# from detectron2.data import DatasetCatalog
# for d in ["Train"]:
#     DatasetCatalog.register("Chart_Dataset_" + d ,  lambda d=d:get_charts_dicts(paths_Train))

In [ ]:
# print(get_charts_dicts(paths_Train))

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_1x.yaml"))
cfg.MASK_ON = False
#print(cfg)

cfg.DATASETS.TRAIN = ("Chart_Dataset_Train",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
#TBD
#cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_   /137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


In [ ]:
# def evaluate_model(model,validation_load):
#   model.eval()
#   main_prediction = 0
#   toatal = 0
#   device = DEVICE
#   with torch.no_grad():
#     for i in validation_load:
#       images,label = i
#       images = images.to(device)
#       label = label.to(device)
#       total += label.size(0)
#       main_o = model(images)
      
#       _,predicted = torch.max(main_o.i,1)

#       ayyd_c  += (predicted==label).sum().item() 

#     acc = ayyd_c/total

#   print("Accuracy is {}".format(acc))

In [ ]:
import json

In [ ]:
 json_fd = open('/content/ICPR2020_CHARTINFO_UB_PMC_TRAIN_v1.21/annotations_JSON/horizontal_bar/PMC4684910___12889_2015_2565_Fig2_HTML.json')

In [ ]:
json_data = json.load(json_fd)

In [ ]:
if

In [ ]:
if('task6' in json_data):
  print(5)